# Simplified Keras/QKeras PiDiNet for hls4ml

**Objective:** Build, train (float and quantized), and prepare a simplified Keras PiDiNet for hls4ml synthesis.

**Motivation:** The original PyTorch PiDiNet implementation uses custom Pixel Difference Convolution (PDC) operations that are not directly supported by standard deep learning frameworks like Keras or quantization toolkits like QKeras, nor by hardware synthesis tools like hls4ml. To deploy PiDiNet on FPGAs using hls4ml, we need a version representable with standard layers.

**PiDiNet Philosophy:** The original PiDiNet workflow involves training a model with PDC operators and then *converting* the learned weights into equivalent standard convolutional kernels before deployment. This conversion captures the edge-detection properties learned by the PDC operators within standard convolution weights.

**Keras/hls4ml Approach:** Due to framework limitations (lack of direct PDC support in Keras/QKeras, hls4ml compatibility), we will adopt a slightly different approach. Instead of training with PDC and then converting, we will **build the Keras/QKeras models directly using the target vanilla CNN structure**. The conversion logic from the original PDC types (cv, cd, ad, rd) will be incorporated by choosing the appropriate `kernel_size` and `padding` for standard `Conv2D`/`QConv2D` layers during the model definition phase. This effectively pre-converts the *architecture* to its standard CNN equivalent before training.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
import qkeras
from qkeras import QConv2D, QActivation, QDense, quantized_bits
import hls4ml
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import tensorflow_datasets as tfds # Example dataset loader

print(f"TensorFlow version: {tf.__version__}")
print(f"QKeras version: {qkeras.__version__}")
print(f"hls4ml version: {hls4ml.__version__}")

# Configure GPU (optional)
# IMPORTANT: After installing/changing TensorFlow version (e.g., downgrading),
# re-run this cell and carefully check its output to ensure the GPU is still detected.
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configure memory growth (good practice)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        # Check if GPUs were found
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU detected. Check TensorFlow installation and CUDA compatibility.")
    print("Note: QKeras training in Cell 6 requires a GPU due to grouped convolutions.")

## Cell 2: Define Keras PiDiNet Architecture (Vanilla Structure)

This model uses standard Keras layers (`Conv2D`, `BatchNormalization`, etc.). The key is that the parameters for the depthwise-like `Conv2D` layers (kernel size, padding) are chosen based on the *original* PDC type they represent, effectively building the converted structure directly.

In [ ]:
# --- Constants ---
NUM_STAGES = 2 # Simplified example
BLOCKS_PER_STAGE = 2 # Simplified example
INITIAL_FILTERS = 16 # Corresponds to 'C' in PiDiNet paper for micro/tiny
DIL_FACTOR = 4 # Example dilation factor if CDCM were used
USE_SA = True # Example flag for CSAM
USE_DIL = True # Example flag for CDCM

# Simplified PDC config list for 2 stages, 2 blocks each + init_block
# Example: ['cd', 'cd', 'cd', 'ad', 'ad'] # init + 2*2 blocks
PDC_CONFIG = ['cd', 'cd', 'cd', 'ad', 'ad']
INPUT_SHAPE = (128, 128, 3) # Example input shape
NUM_CLASSES = 1 # For edge detection (binary)

# --- Helper Function: Get Standard Conv Params from PDC Type ---
def get_pdc_params(original_pdc_type):
    """Maps original PDC type to kernel_size/padding for standard Conv2D."""
    if original_pdc_type == 'rd':
        return {'kernel_size': 5, 'padding': 2}
    elif original_pdc_type in ['cv', 'cd', 'ad']:
        return {'kernel_size': 3, 'padding': 1}
    else:
        raise ValueError(f"Unknown PDC type: {original_pdc_type}")

# --- Keras PDC Block (Converted Structure) ---
def PDCBlock_converted_keras(x, original_pdc_type, inplane, ouplane, stride=1):
    """Implements the PDC block using standard Keras layers."""
    identity = x

    # Shortcut connection
    if stride > 1:
        x = layers.MaxPooling2D(pool_size=2, strides=2)(x)
        identity = layers.Conv2D(ouplane, kernel_size=1, strides=1, padding='same', use_bias=False)(x)
    elif inplane != ouplane:
        identity = layers.Conv2D(ouplane, kernel_size=1, strides=1, padding='same', use_bias=False)(x)
    # else: identity remains x (stride=1, same channels)

    # Main path
    pdc_params = get_pdc_params(original_pdc_type)
    # Depthwise-like conv based on PDC type
    y = layers.Conv2D(inplane, # Depthwise uses input channels
                       kernel_size=pdc_params['kernel_size'],
                       strides=1, # Stride applied in MaxPool
                       padding=pdc_params['padding'],
                       groups=inplane, # Simulate depthwise
                       use_bias=False)(x)
    # Note: Original PiDiNet might have BN/ReLU here, simplified for clarity
    y = layers.ReLU()(y)
    # Pointwise conv
    y = layers.Conv2D(ouplane, kernel_size=1, strides=1, padding='same', use_bias=False)(y)
    # Note: Original PiDiNet might have BN/ReLU here too

    # Residual connection
    out = layers.add([identity, y])
    # Optional: Add final ReLU/BN if needed
    # out = layers.ReLU()(out)
    return out

# --- Define Simplified Keras PiDiNet Model ---
def build_simplified_keras_pidinet(input_shape, pdc_config, initial_filters, num_stages, blocks_per_stage, num_classes):
    x_in = layers.Input(shape=input_shape)
    pdc_idx = 0

    # Initial Block
    init_pdc_type = pdc_config[pdc_idx]
    pdc_params = get_pdc_params(init_pdc_type)
    x = layers.Conv2D(initial_filters,
                      kernel_size=pdc_params['kernel_size'],
                      strides=1,
                      padding=pdc_params['padding'],
                      use_bias=False)(x_in)
    # x = layers.BatchNormalization()(x) # Add BN/ReLU if mimicking full structure
    # x = layers.ReLU()(x)
    pdc_idx += 1

    current_filters = initial_filters
    stage_outputs = []

    # Stages and Blocks
    for stage in range(num_stages):
        for block in range(blocks_per_stage):
            stride = 2 if stage > 0 and block == 0 else 1
            output_filters = current_filters * 2 if stride > 1 else current_filters
            
            if pdc_idx >= len(pdc_config):
                 raise IndexError(f"PDC config list too short. Need at least {pdc_idx+1} entries.")
            
            x = PDCBlock_converted_keras(x, pdc_config[pdc_idx], current_filters, output_filters, stride)
            current_filters = output_filters
            pdc_idx += 1
        stage_outputs.append(x)

    # Fusion Layers (Simplified Example)
    fused_outputs = []
    target_h, target_w = input_shape[0], input_shape[1]

    for i, stage_out in enumerate(stage_outputs):
        # Simplified: Reduce channels, then upsample
        # In full PiDiNet, CDCM/CSAM might be here
        reduced = layers.Conv2D(1, kernel_size=1, padding='same', use_bias=False, name=f'reduce_{i}')(stage_out)
        
        # Upsample to original input size
        # Check if upsampling is needed
        _, h, w, _ = reduced.shape
        if h != target_h or w != target_w:
             upsampled = layers.UpSampling2D(size=(target_h // h, target_w // w), interpolation='bilinear', name=f'upsample_{i}')(reduced)
        else:
             upsampled = reduced
        fused_outputs.append(upsampled)

    # Concatenate side outputs
    if len(fused_outputs) > 1:
        concatenated = layers.Concatenate(axis=-1)(fused_outputs)
    else:
        concatenated = fused_outputs[0]

    # Final Classifier
    output = layers.Conv2D(num_classes, kernel_size=1, padding='same', use_bias=False, name='classifier')(concatenated)
    output = layers.Activation('sigmoid', name='output_sigmoid')(output)

    model = models.Model(inputs=x_in, outputs=output, name='simplified_keras_pidinet')
    return model

# --- Instantiate and Verify ---
keras_model = build_simplified_keras_pidinet(INPUT_SHAPE, PDC_CONFIG, INITIAL_FILTERS, NUM_STAGES, BLOCKS_PER_STAGE, NUM_CLASSES)
keras_model.summary()

# Verify layer sizes (optional but recommended)
print("\nChecking Layer Sizes (Trainable Parameters):")
for layer in keras_model.layers:
    if layer.count_params() > 0:
        layersize = layer.count_params()
        print(f"{layer.name}: {layersize}")
        # Example check for hls4ml latency strategy limit
        # Note: This check is approximate. Actual hls4ml limits depend on specific layer types and configurations.
        # The 4096 limit often applies to weights * activations product in dense/conv.
        # Parameter count is a rough proxy.
        if layersize > 4096*10: # Using a larger proxy limit for parameter count
            print(f"  WARNING: Layer {layer.name} might be large ({layersize} params) for hls4ml 'Latency' strategy.")

## Cell 3: Data Loading and Preprocessing

Load a suitable dataset (like BSDS500) and preprocess it. Ensure labels are binary (0 or 1) and match the model output shape (H, W, 1).

In [ ]:
# --- Data Loading Parameters ---
BATCH_SIZE = 16
BUFFER_SIZE = 1000
# Using a dummy dataset generator for demonstration
# Replace with actual tfds loading (e.g., 'bsds500') or custom data pipeline

def dummy_data_generator(num_samples, img_shape, num_classes):
    """Generates dummy image and mask data."""
    for _ in range(num_samples):
        img = tf.random.uniform(img_shape, minval=0, maxval=1)
        # Generate binary mask
        mask = tf.random.uniform(img_shape[:2] + (num_classes,), minval=0, maxval=2, dtype=tf.int32)
        mask = tf.cast(mask, tf.float32)
        yield img, mask

def preprocess(image, label):
    # Normalize image if not already done
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Ensure label is float32 and has channel dimension
    label = tf.cast(label, tf.float32)
    if len(label.shape) == 2:
        label = label[..., tf.newaxis]
    return image, label

# --- Create Datasets ---
output_signature = (
    tf.TensorSpec(shape=INPUT_SHAPE, dtype=tf.float32),
    tf.TensorSpec(shape=INPUT_SHAPE[:2] + (NUM_CLASSES,), dtype=tf.float32)
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: dummy_data_generator(100, INPUT_SHAPE, NUM_CLASSES), # 100 dummy train samples
    output_signature=output_signature
)
train_dataset = train_dataset.map(preprocess).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_generator(
    lambda: dummy_data_generator(20, INPUT_SHAPE, NUM_CLASSES), # 20 dummy validation samples
    output_signature=output_signature
)
val_dataset = val_dataset.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    lambda: dummy_data_generator(20, INPUT_SHAPE, NUM_CLASSES), # 20 dummy test samples
    output_signature=output_signature
)
test_dataset = test_dataset.map(preprocess).batch(BATCH_SIZE)

print(f"Train dataset: {train_dataset.element_spec}")
print(f"Validation dataset: {val_dataset.element_spec}")
print(f"Test dataset: {test_dataset.element_spec}")

## Cell 4: Floating-Point Model Training

In [ ]:
# --- Training Parameters ---
EPOCHS = 5 # Use more epochs for real training
LEARNING_RATE = 0.001

# --- Compile and Train ---
keras_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                    loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=['accuracy']) # Use appropriate metrics for edge detection (e.g., IoU, F1-score)

print("\nTraining Keras float model...")
history = keras_model.fit(train_dataset,
                          epochs=EPOCHS,
                          validation_data=val_dataset)

# --- Plot History ---
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.tight_layout()
plt.show()

# --- Save Model ---
keras_model.save('simplified_pidinet_float.h5')
print("Saved Keras float model to simplified_pidinet_float.h5")

## Cell 5: Define QKeras PiDiNet Architecture (Vanilla Structure)

This model mirrors the Keras float model structure but uses QKeras layers (`QConv2D`, `QActivation`). The `kernel_size` and `padding` for `QConv2D` are again determined by the original PDC type. We specify quantizers for weights and activations.

In [ ]:
from qkeras import autoqkeras
# Removed incorrect import: from qkeras.qtools.quantized_operators import QuantAdd
# --- Quantizers ---
# Example: 6-bit weights, 6-bit activations
WEIGHT_QUANTIZER = "quantized_bits(6,0,alpha=1)"
ACT_QUANTIZER = "quantized_relu(6)" # ReLU activation
BIAS_QUANTIZER = "quantized_bits(6,0,alpha=1)" # If using bias
REQUANT_QUANTIZER = "quantized_bits(6)" # Generic quantizer for intermediate steps

# --- QKeras PDC Block (Converted Structure) ---
def QuantPDCBlock_keras(x, original_pdc_type, inplane, outplane, stride=1, weight_quantizer=None, act_quantizer=None, bias_quantizer=None):
    """Implements the PDC block using standard QKeras layers."""
    identity = x

    # Shortcut connection
    if stride > 1:
        x_pooled = layers.MaxPooling2D(pool_size=2, strides=2)(x) # Outputs float
        # Requantize after MaxPool before shortcut conv
        x_requant = QActivation(REQUANT_QUANTIZER, name=f'requant_pool_{x.name.split("/")[0]}')(x_pooled)
        identity = QConv2D(outplane, kernel_size=1, strides=1, padding='same',
                           kernel_quantizer=weight_quantizer,
                           bias_quantizer=bias_quantizer,
                           use_bias=False)(x_requant) # Use requantized input
        x = x_requant # Use requantized tensor for main path as well
    elif inplane != outplane:
        identity = QConv2D(outplane, kernel_size=1, strides=1, padding='same',
                           kernel_quantizer=weight_quantizer,
                           bias_quantizer=bias_quantizer,
                           use_bias=False)(x)
    # else: identity remains x (already QuantTensor or requantized)

    # Main path
    pdc_params = get_pdc_params(original_pdc_type)
    # Depthwise-like conv
    y = QConv2D(inplane,
                kernel_size=pdc_params['kernel_size'],
                strides=1,
                padding=pdc_params['padding'],
                groups=inplane,
                kernel_quantizer=weight_quantizer,
                bias_quantizer=bias_quantizer,
                use_bias=False)(x) # Input x is now always QuantTensor or requantized
    y = QActivation(act_quantizer)(y)
    # Pointwise conv
    y = QConv2D(outplane, kernel_size=1, strides=1, padding='same',
                kernel_quantizer=weight_quantizer,
                bias_quantizer=bias_quantizer,
                use_bias=False)(y)
    # Apply activation BEFORE residual add
    y_activated = QActivation(act_quantizer)(y)

    # Residual connection: Wrap standard add in QActivation
    # This ensures the output of the addition is immediately quantized
    out = QActivation(REQUANT_QUANTIZER, name=f'requant_add_{x.name.split("/")[0]}')(
        layers.add([identity, y_activated])
    )

    # Removed redundant QActivation that was previously here
    return out

# --- Define Simplified QKeras PiDiNet Model ---
def build_simplified_qkeras_pidinet(input_shape, pdc_config, initial_filters, num_stages, blocks_per_stage, num_classes,
                                    weight_quantizer, act_quantizer, bias_quantizer):
    x_in = layers.Input(shape=input_shape)
    pdc_idx = 0

    # Input Quantization (optional, depends on hls4ml config)
    # x = QActivation('quantized_bits(8,0)')(x_in)
    x = x_in # Assume input is float or handled by hls4ml

    # Initial Block
    init_pdc_type = pdc_config[pdc_idx]
    pdc_params = get_pdc_params(init_pdc_type)
    # Add input quantizer before first QConv2D if input is float
    x_quant_in = QActivation(REQUANT_QUANTIZER, name='quant_input')(x)
    x = QConv2D(initial_filters,
                kernel_size=pdc_params['kernel_size'],
                strides=1,
                padding=pdc_params['padding'],
                kernel_quantizer=weight_quantizer,
                bias_quantizer=bias_quantizer,
                use_bias=False)(x_quant_in)
    x = QActivation(act_quantizer)(x)
    pdc_idx += 1

    current_filters = initial_filters
    stage_outputs = []

    # Stages and Blocks
    for stage in range(num_stages):
        for block in range(blocks_per_stage):
            stride = 2 if stage > 0 and block == 0 else 1
            output_filters = current_filters * 2 if stride > 1 else current_filters

            if pdc_idx >= len(pdc_config):
                 raise IndexError(f"PDC config list too short. Need at least {pdc_idx+1} entries.")

            x = QuantPDCBlock_keras(x, pdc_config[pdc_idx], current_filters, output_filters, stride,
                                    weight_quantizer, act_quantizer, bias_quantizer)
            current_filters = output_filters
            pdc_idx += 1
        stage_outputs.append(x)

    # Fusion Layers (Simplified Example)
    fused_outputs_requant = []
    target_h, target_w = input_shape[0], input_shape[1]

    for i, stage_out in enumerate(stage_outputs):
        reduced = QConv2D(1, kernel_size=1, padding='same',
                          kernel_quantizer=weight_quantizer,
                          bias_quantizer=bias_quantizer,
                          use_bias=False, name=f'qreduce_{i}')(stage_out)

        # Upsample - standard Keras layer, output is float
        _, h, w, _ = reduced.shape
        if h != target_h or w != target_w:
             upsampled_float = layers.UpSampling2D(size=(target_h // h, target_w // w), interpolation='bilinear', name=f'upsample_{i}')(reduced)
        else:
             upsampled_float = reduced # Still a QuantTensor if not upsampled
        # Requantize after upsampling (or if not upsampled, ensure it's QuantTensor)
        upsampled_requant = QActivation(REQUANT_QUANTIZER, name=f'requant_upsample_{i}')(upsampled_float)
        fused_outputs_requant.append(upsampled_requant)

    # Concatenate side outputs (inputs are now QuantTensors)
    if len(fused_outputs_requant) > 1:
        # Concatenate might still output float if inputs have different scales
        concatenated_maybe_float = layers.Concatenate(axis=-1)(fused_outputs_requant)
    else:
        concatenated_maybe_float = fused_outputs_requant[0]

    # Requantize after Concatenate before classifier
    concatenated_requant = QActivation(REQUANT_QUANTIZER, name='requant_concat')(concatenated_maybe_float)

    # Final Classifier
    output = QConv2D(num_classes, kernel_size=1, padding='same',
                     kernel_quantizer=weight_quantizer,
                     bias_quantizer=bias_quantizer,
                     use_bias=False, name='qclassifier')(concatenated_requant) # Input is requantized
    # Final activation - Use QActivation sigmoid
    output = QActivation('quantized_sigmoid(6)', name='output_sigmoid')(output)

    model = models.Model(inputs=x_in, outputs=output, name='simplified_qkeras_pidinet')
    return model

# --- Instantiate and Verify ---
qkeras_model = build_simplified_qkeras_pidinet(INPUT_SHAPE, PDC_CONFIG, INITIAL_FILTERS, NUM_STAGES, BLOCKS_PER_STAGE, NUM_CLASSES,
                                             WEIGHT_QUANTIZER, ACT_QUANTIZER, BIAS_QUANTIZER)
qkeras_model.summary()

print("\nQKeras Model Quantization Summary:")
autoqkeras.utils.print_qmodel_summary(qkeras_model)

In [ ]:
# --- Test Cell for QuantPDCBlock_keras ---
print("\n--- Testing QuantPDCBlock_keras in Isolation (Identity vs Main Path) ---")

# Check for GPU
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

# Test Parameters (remain the same)
test_batch_size = 2
test_h, test_w = 32, 32
test_in_channels = 16
test_out_channels_same = 16 # For stride=1, same channels
test_out_channels_diff = 32 # For stride=2, different channels
test_stride_1 = 1
test_stride_2 = 2
test_pdc_type = 'cd'

# Dummy Input Data (Float)
dummy_input_shape = (test_batch_size, test_h, test_w, test_in_channels)
dummy_input_data = tf.random.normal(dummy_input_shape)
print(f"Dummy Input Shape: {dummy_input_data.shape}")

# --- Test 1: Identity Path Only ---
print("\n--- Testing Identity Path Only ---")
# Case 1a: Stride = 1, Same Channels (Identity should be just input quantization)
input_tensor_id1a = layers.Input(shape=(test_h, test_w, test_in_channels))
quant_input_id1a = QActivation(REQUANT_QUANTIZER, name='id1a_quant_input')(input_tensor_id1a)
# In QuantPDCBlock, if stride=1 and channels same, identity = x (which is already quant)
# Replace qkeras.QuantizedIdentity with standard Keras Identity
identity_output_1a = layers.Identity(name='id1a_identity_passthrough')(quant_input_id1a)
test_model_identity_1a = models.Model(inputs=input_tensor_id1a, outputs=identity_output_1a, name='IdentityPath_S1_SameC')
# Use explicit shape for dummy labels
dummy_labels_1a_shape = (test_batch_size, test_h, test_w, test_in_channels)
dummy_labels_1a = tf.random.uniform(dummy_labels_1a_shape, minval=0, maxval=1)

# Case 1b: Stride = 1, Different Channels
input_tensor_id1b = layers.Input(shape=(test_h, test_w, test_in_channels))
quant_input_id1b = QActivation(REQUANT_QUANTIZER, name='id1b_quant_input')(input_tensor_id1b)
# Replace QConv2D with standard Conv2D for gradient check
identity_output_1b = layers.Conv2D(test_out_channels_diff, kernel_size=1, strides=1, padding='same',
                                  # kernel_quantizer=WEIGHT_QUANTIZER,
                                  # bias_quantizer=BIAS_QUANTIZER,
                                  use_bias=False, name='id1b_shortcut_conv')(quant_input_id1b)
test_model_identity_1b = models.Model(inputs=input_tensor_id1b, outputs=identity_output_1b, name='IdentityPath_S1_DiffC')
# Use explicit shape for dummy labels
dummy_labels_1b_shape = (test_batch_size, test_h, test_w, test_out_channels_diff)
dummy_labels_1b = tf.random.uniform(dummy_labels_1b_shape, minval=0, maxval=1)

# Case 1c: Stride = 2 (Implies Different Channels)
input_tensor_id1c = layers.Input(shape=(test_h, test_w, test_in_channels))
quant_input_id1c = QActivation(REQUANT_QUANTIZER, name='id1c_quant_input')(input_tensor_id1c)
x_pooled_id1c = layers.MaxPooling2D(pool_size=2, strides=2, name='id1c_pool')(quant_input_id1c)
x_requant_id1c = QActivation(REQUANT_QUANTIZER, name='id1c_requant_pool')(x_pooled_id1c)
# Replace QConv2D with standard Conv2D for gradient check
identity_output_1c = layers.Conv2D(test_out_channels_diff, kernel_size=1, strides=1, padding='same',
                                  # kernel_quantizer=WEIGHT_QUANTIZER,
                                  # bias_quantizer=BIAS_QUANTIZER,
                                  use_bias=False, name='id1c_shortcut_conv')(x_requant_id1c)
test_model_identity_1c = models.Model(inputs=input_tensor_id1c, outputs=identity_output_1c, name='IdentityPath_S2_DiffC')
# Use explicit shape for dummy labels (consider pooling effect)
dummy_labels_1c_shape = (test_batch_size, test_h // 2, test_w // 2, test_out_channels_diff)
dummy_labels_1c = tf.random.uniform(dummy_labels_1c_shape, minval=0, maxval=1)

# Function to run gradient check
def check_gradients(model_to_check, input_data, label_data, model_name):
    print(f"\n--- Gradient Check for: {model_name} ---")
    model_to_check.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='mse', run_eagerly=True)
    with tf.GradientTape() as tape:
        predictions = model_to_check(input_data, training=True)
        loss = model_to_check.compiled_loss(label_data, predictions, regularization_losses=model_to_check.losses)
    print(f"Loss: {loss}")
    if loss is None or tf.math.is_nan(loss):
        print("Error: Loss is None or NaN.")
        return
    trainable_vars = model_to_check.trainable_variables
    if not trainable_vars:
        print("Warning: No trainable variables found.")
        return
    gradients = tape.gradient(loss, trainable_vars)
    print("Gradients:")
    all_none = True
    none_vars = []
    for var, grad in zip(trainable_vars, gradients):
        if grad is not None:
            print(f"  Var: {var.name}, Grad Shape: {grad.shape}")
            all_none = False
        else:
            print(f"  Var: {var.name}, Grad: None")
            none_vars.append(var.name)
    if all_none:
        print("Result: All gradients are None.")
    elif none_vars:
        print(f"Result: Some gradients are None: {none_vars}")
    else:
        print("Result: All gradients computed.")

# Run checks for identity paths
check_gradients(test_model_identity_1a, dummy_input_data, dummy_labels_1a, test_model_identity_1a.name)
check_gradients(test_model_identity_1b, dummy_input_data, dummy_labels_1b, test_model_identity_1b.name)
check_gradients(test_model_identity_1c, dummy_input_data, dummy_labels_1c, test_model_identity_1c.name)

# --- Test 2: Main Path Only ---
print("\n--- Testing Main Path Only ---")
pdc_params = get_pdc_params(test_pdc_type) # Use helper from Cell 2
input_tensor_main = layers.Input(shape=(test_h, test_w, test_in_channels))
quant_input_main = QActivation(REQUANT_QUANTIZER, name='main_quant_input')(input_tensor_main)
# Replace QConv2D with standard Conv2D for gradient check
y_main = layers.Conv2D(test_in_channels, kernel_size=1, strides=1, padding='same', # DEBUG 1x1
                      # groups=test_in_channels, # Grouped conv commented out
                      # kernel_quantizer=WEIGHT_QUANTIZER,
                      # bias_quantizer=BIAS_QUANTIZER,
                      use_bias=False, name='main_conv1')(quant_input_main)
y_main = QActivation(ACT_QUANTIZER, name='main_act1')(y_main)
# Replace QConv2D with standard Conv2D for gradient check
y_main = layers.Conv2D(test_out_channels_same, kernel_size=1, strides=1, padding='same',
                      # kernel_quantizer=WEIGHT_QUANTIZER,
                      # bias_quantizer=BIAS_QUANTIZER,
                      use_bias=False, name='main_conv2')(y_main)
main_path_output = QActivation(ACT_QUANTIZER, name='main_act2')(y_main)
test_model_main = models.Model(inputs=input_tensor_main, outputs=main_path_output, name='MainPath')
# Use explicit shape for dummy labels
dummy_labels_main_shape = (test_batch_size, test_h, test_w, test_out_channels_same)
dummy_labels_main = tf.random.uniform(dummy_labels_main_shape, minval=0, maxval=1)

# Run check for main path
check_gradients(test_model_main, dummy_input_data, dummy_labels_main, test_model_main.name)

print("\n--- Isolation Test Complete ---")

## Cell 6: Quantization-Aware Training (QAT)

**Important Note:** Training this specific QKeras model requires a **GPU**. The `QuantPDCBlock_keras` uses `QConv2D` with `groups=inplane` to mimic depthwise convolutions. TensorFlow's CPU backend does not support gradient calculations for grouped convolutions, leading to an `InvalidArgumentError`. Ensure a GPU is available and configured correctly (as attempted in Cell 1) before running the training cell below.

In [ ]:
# --- Compile and Train QKeras Model ---
qkeras_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                     loss=tf.keras.losses.BinaryCrossentropy(),
                     metrics=['accuracy'],
                     run_eagerly=True) # Set run_eagerly=True if encountering graph mode issues

print("\nTraining QKeras model...")
# Check if GPU is available before training
if not tf.config.list_physical_devices('GPU'):
    print("\nWARNING: No GPU detected. QKeras training with grouped convolutions requires a GPU.")
    print("Skipping QKeras training.")
    # Optionally, load a pre-trained model or skip subsequent steps
    # qkeras_model = qkeras.utils.load_qmodel('path/to/pretrained_qkeras.h5')
else:
    q_history = qkeras_model.fit(train_dataset,
                               epochs=EPOCHS,
                               validation_data=val_dataset)

    # --- Plot History ---
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(q_history.history['loss'], label='Training Loss')
    plt.plot(q_history.history['val_loss'], label='Validation Loss')
    plt.title('QKeras Loss')
    plt.xlabel('Epoch')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(q_history.history['accuracy'], label='Training Accuracy')
    plt.plot(q_history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('QKeras Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # --- Save Model ---
    qkeras_model.save('simplified_pidinet_qkeras.h5')
    print("Saved QKeras model to simplified_pidinet_qkeras.h5")

## Cell 7: Model Evaluation

In [ ]:
# --- Load Models ---
print("\nLoading models for evaluation...")
loaded_keras_model = tf.keras.models.load_model('simplified_pidinet_float.h5')

# Need custom objects for QKeras layers - Use load_qmodel instead
# custom_objects = qkeras.utils.get_quantizer_custom_objects()
# custom_objects.update(qkeras.utils.get_auto_custom_objects())
# loaded_qkeras_model = tf.keras.models.load_model('simplified_pidinet_qkeras.h5', custom_objects=custom_objects)
loaded_qkeras_model = qkeras.utils.load_qmodel('simplified_pidinet_qkeras.h5')

print("Evaluating Keras float model...")
loss_float, acc_float = loaded_keras_model.evaluate(test_dataset, verbose=0)
print(f"  Float Model - Loss: {loss_float:.4f}, Accuracy: {acc_float:.4f}")

print("Evaluating QKeras model...")
# Evaluate the loaded QKeras model
loss_q, acc_q = loaded_qkeras_model.evaluate(test_dataset, verbose=0)
print(f"  QKeras Model - Loss: {loss_q:.4f}, Accuracy: {acc_q:.4f}")

# --- Visualize Predictions (Example) ---
print("\nVisualizing predictions...")
for images, labels in test_dataset.take(1):
    pred_float = loaded_keras_model.predict(images)
    pred_q = loaded_qkeras_model.predict(images)

    plt.figure(figsize=(12, 6))
    n_show = min(BATCH_SIZE, 4)
    for i in range(n_show):
        # Original Image
        plt.subplot(3, n_show, i + 1)
        plt.imshow(images[i])
        plt.title("Input")
        plt.axis('off')

        # Float Prediction
        plt.subplot(3, n_show, i + 1 + n_show)
        plt.imshow(pred_float[i][..., 0], cmap='gray') # Assuming single channel output
        plt.title("Float Pred")
        plt.axis('off')

        # QKeras Prediction
        plt.subplot(3, n_show, i + 1 + 2 * n_show)
        plt.imshow(pred_q[i][..., 0], cmap='gray')
        plt.title("QKeras Pred")
        plt.axis('off')

    plt.tight_layout()
    plt.show()
    break

### Strategy: Training on Newer TF, Converting on Older TF

If you encounter GPU compatibility issues with a specific TensorFlow version required for a later step (like hls4ml conversion), you can try the following:

1.  **Train:** Use an environment (e.g., default Colab) with a newer TensorFlow version where the GPU works correctly. Train your model (float or QKeras) and save it using `model.save('my_model_trained.h5')`.
2.  **Switch Environment:** Start a new environment and install the required older TensorFlow version (e.g., `pip install "tensorflow==2.14.0"`). Also install the *exact same versions* of QKeras and any other relevant libraries (like `tensorflow-model-optimization`) used during training.
3.  **Load:** Load the saved model in the older TF environment using `tf.keras.models.load_model('my_model_trained.h5', custom_objects=...)`. Remember to provide the necessary `custom_objects` for QKeras/pruning layers.
4.  **Convert/Synthesize:** Proceed with the steps requiring the older TF version (e.g., hls4ml conversion below).

**Caveats:**
*   **Forward Compatibility:** Loading a model saved in a newer TF version into an older one might fail if the model uses features not present in the older version.
*   **Dependency Versions:** Ensure library versions (QKeras, etc.) are compatible across both TF environments.

## Cell 8: hls4ml Conversion

Now, we convert the trained QKeras model to an hls4ml project. 

**Important:** Because the QKeras model was constructed using the target vanilla CNN structure (with kernel sizes/padding reflecting the original PDC conversion logic), no separate weight/model conversion step (like the original PiDiNet's `convert_pidinet.py`) is required before hls4ml conversion.

In [ ]:
# --- Load Trained QKeras Model ---
print("\nLoading QKeras model for hls4ml conversion...")
custom_objects = qkeras.utils.get_quantizer_custom_objects()
custom_objects.update(qkeras.utils.get_auto_custom_objects())
qmodel_for_hls = tf.keras.models.load_model('simplified_pidinet_qkeras.h5', custom_objects=custom_objects)

# --- Define hls4ml Configuration ---
hls_config = hls4ml.utils.config_from_keras_model(qmodel_for_hls,
                                                 granularity='name', # Layer-by-layer config
                                                 backend='Vitis', # Or Vivado
                                                 # Default precision if not inferred from QKeras:
                                                 # default_precision='ap_fixed<16,6>'
                                                )

# **Crucial for CNNs:** Use streaming IO
hls_config['IOType'] = 'io_stream'

# Set Strategy (Latency for max parallelism, Resource for lower resources)
# Check layer sizes from Cell 2 - if any are very large, 'Resource' might be needed
hls_config['Model']['Strategy'] = 'Latency'

# Specify target device (Update with your actual device)
XILINX_PART = 'xcu250-figd2104-2L-e' # Example UltraScale+
# XILINX_PART = 'xcvu9p-flgb2104-2L-e'
# XILINX_PART = 'xc7vx690t-ffg1761-2' # Example Virtex-7

# --- Print and Modify Config (Optional) ---
print("\nGenerated hls4ml config:")
# hls4ml.utils.print_dict(hls_config)

# Example modification: Increase ReuseFactor for a specific large layer if needed
# if 'qconv2d_large_layer_name' in hls_config['LayerName']:
#    hls_config['LayerName']['qconv2d_large_layer_name']['ReuseFactor'] = 4
#    hls_config['LayerName']['qconv2d_large_layer_name']['Strategy'] = 'Resource'

# --- Convert to hls4ml Model ---
output_dir = 'hls4ml_simplified_pidinet_prj'
print(f"\nConverting to hls4ml project in: {output_dir}")

hls_model = hls4ml.converters.convert_from_keras_model(
    qmodel_for_hls,
    hls_config=hls_config,
    output_dir=output_dir,
    backend='Vitis', # Match backend in config
    part=XILINX_PART
)

# --- Compile the hls4ml Model ---
# This generates the HLS C++ code
print("\nCompiling hls4ml model...")
hls_model.compile()
print("Compilation complete.")

# --- Optional: Build (Synthesize) the Project ---
# This runs Vivado/Vitis HLS - can take a long time!
run_synthesis = False
if run_synthesis:
    print("\nRunning HLS synthesis (this may take a while)...")
    # csim=True runs C simulation, synth=True runs C synthesis, cosim=True runs RTL simulation
    # vsynth=True runs Vivado synthesis for resource estimates
    hls_model.build(csim=False, synth=True, cosim=False, validation=False, vsynth=True)
    print("\nSynthesis complete. Reports are in:")
    print(f"{output_dir}/myproject_prj/solution1/syn/report/myproject_csynth.rpt")
    print(f"{output_dir}/vivado_synth.rpt")

    # --- Optional: Print Reports ---
    # hls4ml.report.read_vivado_report(output_dir)
else:
    print("\nSkipping HLS synthesis. Project generated in:", output_dir)